In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [2]:
from stockMarket.core import get_tickers_from_index

tickers = get_tickers_from_index('russell2000')

In [3]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(tickers)
# contracts = populate_contracts(tickers, update_period="now")

### Perform Screening

In [4]:
import numpy as np
import pandas as pd

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
payout_screener = LimitScreenerObject('payout_ratio', max_value=150, description="Payout Ratio")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', min_value=0, max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', min_value=0, max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.min, description="Min Net Income")

screener_list = [
    dividend_yield_screener,
    payout_screener,
    price_screener,
    trailing_pe_screener,
    forward_pe_screener,
    revenue_screener,
    operating_cashflow_screener,
    equity_ratio_screener_not_bank,
    equity_ratio_screener_bank,
    ebitda_margin_screener,
    net_income_screener
]

screened_contracts = screener.screen(screener_list)

/home/jag/Trading/stockMarket/stockMarket/core/api.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  (data[i] - data[-1]) / data[-1] / (len(data) - i - 1) * 100)
/home/jag/Trading/stockMarket/stockMarket/core/api.py:17: RuntimeWarning: invalid value encountered in scalar divide
  (data[i] - data[-1]) / data[-1] / (len(data) - i - 1) * 100)
/home/jag/Trading/stockMarket/stockMarket/core/financialStatement.py:42: RuntimeWarning: divide by zero encountered in divide
  return (self.total_assets - self.total_liabilities) / self.total_assets * 100


### Collect Data

In [5]:
from stockMarket.core import contracts_to_df

df = contracts_to_df(screened_contracts)
df.to_csv("screened_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
df

,Name,Sector,Financial Statement Type,Price,Trailing PE,Forward PE,Market Cap,Dividend Yield,Payout Ratio,Revenue,...,Net Income,Avg Net Income,Min Net Income,Max Net Income,EBIT Margin,EBITDA Margin,Operating Cash Flow,Equity Ratio,Earnings Date,Ex-Dividend Date
ASIX,AdvanSix Inc.,Basic Materials,Industry,26.24,7.690909,17.148647,684.084864,2.52,18.030001,1945.640,...,171.886,102.007333,41.347,171.886,11.653286,15.217820,273.601,49.365659,NaT,NaT
SUM,"Summit Materials, Inc.",Basic Materials,Industry,37.34,13.756654,19.988950,6303.315456,NaN,NaN,2412.522,...,272.145,129.516333,33.906,272.145,18.225699,27.033950,284.098,46.979105,NaT,NaT
TGNA,TEGNA Inc.,Communication Services,Industry,15.75,5.489437,4.492795,3070.731264,2.92,14.039999,3279.245,...,630.469,425.974333,273.744,630.469,30.209149,33.901371,812.151,41.912479,NaT,NaT
TPH,"Tri Pointe Homes, Inc.",Consumer Cyclical,Industry,34.94,8.484029,9.618384,3351.212288,NaN,NaN,4348.533,...,576.060,331.970500,187.191,576.060,17.719700,18.363825,444.276,60.009004,NaT,NaT
ETD,Ethan Allen Interiors Inc.,Consumer Cyclical,Industry,30.73,9.336538,9.613861,739.890304,4.94,44.869998,791.382,...,105.807,56.676833,8.900,105.807,17.336255,19.309259,100.664,63.186814,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMKR,"Amkor Technology, Inc.",Technology,Industry,32.73,19.304878,15.443903,7781.015040,1.00,18.290001,7091.585,...,765.823,376.414333,120.888,765.823,12.644874,21.284720,1098.756,53.780163,NaT,NaT
EXLS,"ExlService Holdings, Inc.",Technology,Industry,32.08,29.790478,19.428572,5158.791680,NaN,NaN,1412.044,...,142.968,86.745833,48.888,142.968,13.608783,17.581888,166.141,56.323327,NaT,NaT
HCKT,"The Hackett Group, Inc.",Technology,Industry,23.45,18.349207,13.761906,629.731008,1.90,34.919998,293.742,...,40.802,27.060000,5.473,41.545,18.808342,19.978416,58.904,31.502814,NaT,NaT
AVA,Avista Corporation,Utilities,Utility,34.38,15.459090,14.112032,2631.271936,5.41,82.730000,1710.207,...,155.176,146.887000,115.916,196.979,11.123917,25.925751,124.207,31.475770,NaT,NaT


### Write Email

In [6]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Screener"
body = screener.screening_details
attachment = "screened_contracts.csv"
write_email(emails, subject, body, attachment)